In [10]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
#from rasterstats import zonal_stats
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import box


# Data Preperation

In [2]:
#Load pre-existing park data
parks_gdf = gpd.read_file('Data/Riyadh_parks_stats_2024.geojson')
print("Loaded parks_gdf with columns:", parks_gdf.columns) 

Loaded parks_gdf with columns: Index(['osm_id', 'OBJECTID', 'FEATURE_ANAME', 'MUNICIPALITY', 'DISTRICT',
       'WALKING_TRACK', 'GREEN_AREAS', 'LAYERID', 'LAYERANAME', 'Validation',
       'Park_id', 'area_m2', 'perimeter_m', 'LSI', 'ndvi_mean',
       'ndvi_pixel_count', 'pisi_mean', 'pisi_pixel_count', 'geometry'],
      dtype='object')


In [3]:
# File link: https://drive.google.com/file/d/1OCiJFoE-HmlpSLjt37WhoBhgVWdKSz0W/view?usp=sharing
# Then it should be added to a folder called Raster within the Data folder

# Define LST file path
lst_2024_path = 'Data/Raster/Riyadh_LST_EPSG20438_2024.tif' 


In [4]:
# Open LST raster to access pixel values and transformation parameters.
with rasterio.open(lst_2024_path) as lst_src:
    lst_transform = lst_src.transform
    lst_crs = lst_src.crs
    lst_data = lst_src.read(1)
   # print(np.nanmin(lst_data), np.nanmax(lst_data))

## 1. Buffer Creation

For each park, createt buffers at 30m intervals up to 300m for the comparison of LST across different zones, providing insights into the urban heat island effect and park cooling influence. ([Cai et al., 2023](https://www.researchgate.net/publication/374556563_Cooling_island_effect_in_urban_parks_from_the_perspective_of_internal_park_landscape); [Zhang et al., 2024](https://www.nature.com/articles/s41598-024-67277-2))


In [5]:
# 2. Generate Buffers (30m intervals up to 300m)
buffer_distances = range(30, 301, 30)  # [30, 60, 90,..., 300]

# Initialize buffer_gdf with a sample row to set dtypes, ensuring Park_id as int
# and to avoid warning i got:  The behavior of DataFrame concatenation with empty or all-NA entries is deprecated
initial_row = {'Park_id': 0, 'distance': 0, 'geometry': Polygon()}
buffer_gdf = gpd.GeoDataFrame([initial_row], crs=parks_gdf.crs)
buffer_gdf = buffer_gdf.iloc[0:0]  # Clear the initial row, keeping dtypes


for park_id, park in parks_gdf.iterrows():
    for dist in buffer_distances:
        # Create donut-shaped buffers (outer - inner)
        outer = park.geometry.buffer(dist)
        inner = park.geometry.buffer(max(0, dist-30))  # Handle 0-30m case
        ring = outer.difference(inner)
        
        # Append to GeoDataFrame
        buffer_gdf = pd.concat([
            buffer_gdf,
            gpd.GeoDataFrame({
                'Park_id': [int(park['Park_id'])],  # Use actual Park_id for accurate mapping
                'distance': [dist],
                'geometry': [ring]
            }, crs=parks_gdf.crs)
        ], ignore_index=True)


In [6]:
# Verify the output structure
print(f"Generated {len(buffer_gdf)} buffer rings")
print("GeoDataFrame columns:", buffer_gdf.columns)
print("CRS:", buffer_gdf.crs)
print("Sample data:")
print(buffer_gdf.head(3))

Generated 2210 buffer rings
GeoDataFrame columns: Index(['Park_id', 'distance', 'geometry'], dtype='object')
CRS: EPSG:20438
Sample data:
   Park_id  distance                                           geometry
0        1        30  POLYGON ((678371.261 2735243.793, 678368.638 2...
1        1        60  POLYGON ((678387.297 2735218.438, 678382.05 27...
2        1        90  POLYGON ((678403.333 2735193.084, 678395.463 2...


In [8]:
# # Save to GeoJSON
# ## Commented out as it's needed once
# buffer_gdf.to_file('Data/Riyadh_parks_buffers.geojson', driver='GeoJSON')
# print("Saved buffers to GeoJSON")

Saved buffers to GeoJSON


## 2. LST Calculation

Here, the pixel-polygon overlap appraoch is used for mean calculation. Check Section 2.2 of '04_FactorsCalculations.ipynb' for justification.

In [12]:
# Initialize buffer columns for LST means and pixel counts in parks_gdf (park interiors) and buffer_gdf (buffer zones)
for dist in range(30, 301, 30):
    parks_gdf[f'lst_buffer_{dist}m'] = np.nan

parks_gdf['lst_park'] = np.nan
parks_gdf['park_pixel_count'] = 0
buffer_gdf['lst_mean'] = np.nan
buffer_gdf['pixel_count'] = 0


### 2.1 Calculate Mean LST within each park

In [14]:
# Calculate LST for park interiors manually with fractional overlap
# Followin NDVI manual method for consistency, weighting pixels >50% within park
with rasterio.open(lst_2024_path) as lst_src:
    # Get raster properties for overlap calculation
    pixel_area = abs(lst_src.transform[0] * lst_src.transform[4])  # 30m x 30m = 900 m²
    threshold_area = 0.50 * pixel_area  # 50% threshold, 450 m²
    raster_nodata = lst_src.nodata if lst_src.nodata is not None else -999

    for idx, park in parks_gdf.iterrows():
        geom = park.geometry
        if not geom.is_valid:
            geom = geom.buffer(0)
            print(f"Repaired invalid geometry for park {park['Park_id']}")

        # Rasterize park mask with all_touched
        mask = rasterio.features.rasterize(
            [(geom, 1)],
            out_shape=(lst_src.height, lst_src.width),
            transform=lst_src.transform,
            fill=0,
            default_value=1,
            all_touched=True
        ) == 1

        # Extract and weight LST values
        lst_park_data = lst_src.read(1)
        y_indices, x_indices = np.where(mask)
        total_weighted_sum = 0
        total_weighted_area = 0

        for y, x in zip(y_indices, x_indices):
            pixel_geom = box(
                lst_src.transform[2] + x * lst_src.transform[0],  # minx
                lst_src.transform[5] + y * lst_src.transform[4],  # miny
                lst_src.transform[2] + (x + 1) * lst_src.transform[0],  # maxx
                lst_src.transform[5] + (y + 1) * lst_src.transform[4]  # maxy
            )
            intersection = geom.intersection(pixel_geom)
            if not intersection.is_empty:
                overlap_area = intersection.area
                if overlap_area >= threshold_area:  # Only include >50% overlap
                    lst_value = lst_park_data[y, x]
                    if lst_value != raster_nodata:
                        total_weighted_sum += lst_value * overlap_area
                        total_weighted_area += overlap_area
                        parks_gdf.at[idx, 'park_pixel_count'] += 1

        # Calculate weighted mean for park interior
        if total_weighted_area > 0:
            parks_gdf.at[idx, 'lst_park'] = total_weighted_sum / total_weighted_area
        else:
            print(f"Park {park['Park_id']} has no valid overlapping pixels.")

Repaired invalid geometry for park 155.0


### 2.2 Calculate Mean LST within each buffer zone for each park

In [15]:
# Calculate LST for buffer zones manually with fractional overlap
# Mirrors NDVI approach, ensuring consistent weighting across buffer distances
with rasterio.open(lst_2024_path) as lst_src:
    # Reuse raster properties for buffer calculations
    pixel_area = abs(lst_src.transform[0] * lst_src.transform[4])  # 30m x 30m = 900 m²
    threshold_area = 0.50 * pixel_area  # 50% threshold, 450 m²
    raster_nodata = lst_src.nodata if lst_src.nodata is not None else -999

    for idx, buffer in buffer_gdf.iterrows():
        geom = buffer.geometry
        park_id = buffer['Park_id']
        dist = buffer['distance']

        if not geom.is_valid:
            geom = geom.buffer(0)
            print(f"Repaired invalid geometry for buffer of park {park_id} at {dist}m")

        # Rasterize buffer mask with all_touched
        mask = rasterio.features.rasterize(
            [(geom, 1)],
            out_shape=(lst_src.height, lst_src.width),
            transform=lst_src.transform,
            fill=0,
            default_value=1,
            all_touched=True
        ) == 1

        # Extract and weight LST values for buffer
        lst_buffer_data = lst_src.read(1)
        y_indices, x_indices = np.where(mask)
        total_weighted_sum = 0
        total_weighted_area = 0

        for y, x in zip(y_indices, x_indices):
            pixel_geom = box(
                lst_src.transform[2] + x * lst_src.transform[0],  # minx
                lst_src.transform[5] + y * lst_src.transform[4],  # miny
                lst_src.transform[2] + (x + 1) * lst_src.transform[0],  # maxx
                lst_src.transform[5] + (y + 1) * lst_src.transform[4]  # maxy
            )
            intersection = geom.intersection(pixel_geom)
            if not intersection.is_empty:
                overlap_area = intersection.area
                if overlap_area >= threshold_area:  # Only include >50% overlap
                    lst_value = lst_buffer_data[y, x]
                    if lst_value != raster_nodata:
                        total_weighted_sum += lst_value * overlap_area
                        total_weighted_area += overlap_area
                        buffer_gdf.at[idx, 'pixel_count'] += 1

        # Calculate weighted mean for buffer zone
        if total_weighted_area > 0:
            buffer_gdf.at[idx, 'lst_mean'] = total_weighted_sum / total_weighted_area
        else:
            print(f"Buffer for park {park_id} at {dist}m has no valid overlapping pixels.")

### 2.3 Appending results to parks_gdf

**Note:** lst_park (interior LST) is calculated and added separately in sec 2.1

In [16]:
# Adding LST buffer values to parks_gdf
for park_id in parks_gdf['Park_id'].unique():
    park_buffers = buffer_gdf[buffer_gdf['Park_id'] == park_id]
    for _, buffer in park_buffers.iterrows():
        dist = buffer['distance']
        lst_value = buffer['lst_mean']
        if lst_value is not None and not np.isnan(lst_value):
            parks_gdf.loc[parks_gdf['Park_id'] == park_id, f'lst_buffer_{dist}m'] = lst_value

print("Processing complete. Results include:")
print(f"- {len(parks_gdf)} parks processed")
print(f"- Cooling effects calculated for buffers 30-300m")

Processing complete. Results include:
- 221 parks processed
- Cooling effects calculated for buffers 30-300m


In [19]:
print(f"- Number of parks with valid lst_park: {parks_gdf['lst_park'].notna().sum()} ({parks_gdf['park_pixel_count'].sum()} pixels)")
print(f"- Number of buffers with valid lst_mean: {buffer_gdf['lst_mean'].notna().sum()} ({buffer_gdf['pixel_count'].sum()} pixels)")

- Number of parks with valid lst_park: 221 (3304 pixels)
- Number of buffers with valid lst_mean: 2210 (148420 pixels)


## 3. PCI - PCE ?

In [10]:
# Calculate cooling effects (ΔT between buffers and park interior)
for dist in range(30, 301, 30):
    parks_gdf[f'cooling_diff_{dist}m'] = (
        parks_gdf[f'lst_buffer_{dist}m'] - parks_gdf['lst_park']
    )

In [11]:
# Comment out export, only needed once:
parks_gdf.to_file('Data/Riyadh_parks_with_LST_Overlap.geojson', driver='GeoJSON')


## 3. Results Exploration

In [ ]:
parks_gdf.head(4)

In [ ]:
parks_gdf.columns

In [ ]:
parks_gdf['max_cooling'] = parks_gdf[[f'cooling_diff_{d}m' for d in range(30, 301, 30)]].min(axis=1)

parks_gdf['cooling_radius'] = (
    parks_gdf[[f'cooling_diff_{d}m' for d in range(30, 301, 30)]]
    .idxmin(axis=1)
    .str.extract(r'(\d+)')
    .astype(int)
)
# Verify results
print(parks_gdf[['park_id', 'lst_park', 'max_cooling', 'cooling_radius']].head())

In [ ]:
# Summary
avg_max_cooling = parks_gdf['max_cooling'].mean()
cooling_park_count = len(parks_gdf[parks_gdf['max_cooling'] < 0])
heat_island_count = len(parks_gdf[parks_gdf['max_cooling'] > 0])
avg_radius = parks_gdf['cooling_radius'].mean()

print(f"""
All praks summary:
1. Average maximum cooling effect: {avg_max_cooling:.1f}°C
2. {cooling_park_count} parks ({cooling_park_count/len(parks_gdf):.0%}) provide cooling
3. {heat_island_count} parks ({heat_island_count/len(parks_gdf):.0%}) act as heat islands  
4. Average cooling radius: {avg_radius:.0f}m
""")

In [ ]:
# Summary statistics for cooling differences
cooling_diff_stats = pd.DataFrame({
    'min': [parks_gdf[f'cooling_diff_{d}m'].min() for d in range(30, 301, 30)],
    'max': [parks_gdf[f'cooling_diff_{d}m'].max() for d in range(30, 301, 30)],
    'mean': [parks_gdf[f'cooling_diff_{d}m'].mean() for d in range(30, 301, 30)],
    'std': [parks_gdf[f'cooling_diff_{d}m'].std() for d in range(30, 301, 30)]
}, index=[f'{d}m' for d in range(30, 301, 30)])
print("\nSummary Statistics for Cooling Differences:")
print(cooling_diff_stats)

The above results show varying cooling/heating impact

In [ ]:
# Get top 5 cooling parks
top_coolers = parks_gdf.nsmallest(5, 'max_cooling')[['park_id', 'impervious_prop','max_cooling', 'cooling_radius', 'area_m2', 'ndvi_mean']]

print("\nTop Cooling Parks:")
print(top_coolers.to_markdown(index=False))

# Calculate their common traits
avg_top_ndvi = top_coolers['ndvi_mean'].mean()
avg_top_area = top_coolers['area_m2'].mean()
avg_heat_paved = top_coolers['impervious_prop'].mean()

print(f"\nCommon Traits:\n- Average NDVI: {avg_top_ndvi:.2f}\n- Average paved surface: {avg_heat_paved:.0%}\n- Average Area: {avg_top_area:,.0f} m²")

In [ ]:
# Analyze worst heat islands
heat_islands = parks_gdf.nlargest(5, 'max_cooling')[['park_id','ndvi_mean', 'max_cooling', 'impervious_prop', 'area_m2']]

print("\nWorst Heat Islands:")
print(heat_islands.to_markdown(index=False))

# Calculate their characteristics
avg_heat_paved = heat_islands['impervious_prop'].mean()
avg_heat_size = heat_islands['area_m2'].mean()
avg_heat_ndvi = heat_islands['ndvi_mean'].mean()

print(f"\nCommon Issues:\n- Average NDVI: {avg_top_ndvi:.2f}\n- Average paved surface: {avg_heat_paved:.0%}\n- Average size: {avg_heat_size:,.0f} m²")

In [ ]:
# Calculate mean cooling at each distance
distances = [0] + list(range(30, 301, 30))
mean_cooling = [0] + [parks_gdf[f'cooling_diff_{d}m'].mean() for d in range(30, 301, 30)]

# Plot
plt.figure(figsize=(8,5))
plt.plot(distances, mean_cooling, color='#2e8b57', linewidth=2, marker='o')
plt.axhline(0, color='gray', linestyle='--')
plt.fill_between(distances, mean_cooling, 0, where=np.array(mean_cooling)<0, 
                 color='#9fd5b3', alpha=0.3)
plt.title("Mean Temperature Differential by Distance from Park Edge", pad=20)
plt.xlabel("Distance (m)")
plt.ylabel("Δ Temperature (°C)")
plt.grid(alpha=0.3)

In [ ]:
# Compare vegetation-rich vs. paved parks
plt.figure(figsize=(10,5))
for group, color in [('High Veg', 'green'), ('Low Veg', 'brown')]:
    subset = parks_gdf[parks_gdf['vegetation_prop'] > 0.5] if group == 'High Veg' else parks_gdf[parks_gdf['vegetation_prop'] <= 0.5]
    means = [subset[f'cooling_diff_{d}m'].mean() for d in range(30,301,30)]
    plt.plot(distances[1:], means, label=group, color=color, marker='o')

plt.axhline(0, color='gray', ls='--')
plt.legend()

In [ ]:
# Plot LST (not ΔT) to see absolute temperatures
plt.figure(figsize=(10,6))
for label, color in [('High Veg', 'green'), ('Low Veg', 'gray')]:
    subset = parks_gdf[parks_gdf['vegetation_prop'] > 0.5] if label == 'High Veg' else parks_gdf[parks_gdf['vegetation_prop'] <= 0.5]
    plt.plot(distances[1:], 
             [subset[f'lst_buffer_{d}m'].mean() for d in range(30,301,30)], 
             label=label, color=color, marker='o')

plt.plot([0, 300], [subset['lst_park'].mean()]*2, '--', color='black', label='Park Interior')
plt.legend()
plt.ylabel("Absolute LST (°C)")

Investigate parks (Identified from QGIS)

In [ ]:
park_212 = parks_gdf[parks_gdf['park_id'] == 212].iloc[0]
print(f"""
Park 212 Profile:
- Name: {park_212['FEATURE_ANAME']}
- Area: {park_212['area_m2']:,.0f} m²
- NDVI: {park_212['ndvi_mean']:.2f}
- Vegetation Cover: {park_212['vegetation_prop']:.0%}
- Impervious: {park_212['impervious_prop']:.0%}
""")

In [ ]:
# Prepare data
distances = range(30, 301, 30)
park_lst = park_212['lst_park']
buffer_lst = [park_212[f'lst_buffer_{d}m'] for d in distances]
cooling = [park_212[f'cooling_diff_{d}m'] for d in distances]

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))

# Absolute temperatures
ax1.plot([0] + list(distances), [park_lst] + buffer_lst, 
         marker='o', color='red')
ax1.set_title(f"Park 212: LST Profile\n(Park Interior = {park_lst:.1f}°C)")
ax1.set_xlabel("Distance from Edge (m)")
ax1.set_ylabel("Land Surface Temperature (°C)")
ax1.grid(alpha=0.3)

# Cooling effect
ax2.plot(distances, cooling, marker='o', color='blue')
ax2.axhline(0, color='gray', ls='--')
ax2.set_title("Cooling Effect (ΔT = Buffer - Park)")
ax2.set_xlabel("Distance from Edge (m)")
ax2.set_ylabel("Δ Temperature (°C)")
ax2.grid(alpha=0.3)

plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt

# Temperature profile
distances = range(0, 301, 30)
temps = [park_212['lst_park']] + [park_212[f'lst_buffer_{d}m'] for d in distances[1:]]

plt.figure(figsize=(10,5))
plt.plot(distances, temps, marker='o', color='darkred')
plt.title(f"Park 212: Thermal Profile\n(Area: {park_212['area_m2']/10000:.1f} ha, NDVI: {park_212['ndvi_mean']:.2f})")
plt.xlabel("Distance from Edge (m)")
plt.ylabel("LST (°C)")
plt.grid()
plt.annotate(f"Max ΔT: +{park_212['cooling_diff_210m']:.1f}°C at 210m", 
             xy=(210, 51.8), xytext=(100, 53),
             arrowprops=dict(facecolor='black'))

In [ ]:
# Compare to typical parks
median_park = parks_gdf[parks_gdf['vegetation_prop'] > 0.6]['lst_park'].median()
print(f"Median LST for vegetated parks: {median_park:.1f}°C vs Park 212: {park_212['lst_park']:.1f}°C")

For COnf

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(parks_gdf['max_cooling'], bins=20, color='skyblue', edgecolor='black')
plt.axvline(parks_gdf['max_cooling'].mean(), color='red', linestyle='--', label=f'Mean: {parks_gdf["max_cooling"].mean():.2f}°C')
plt.title('Distribution of Maximum Cooling Effect')
plt.xlabel('Maximum Cooling Effect (°C)')
plt.ylabel('Number of Parks')
plt.legend()
plt.grid()
plt.show()

In [ ]:
parks_gdf['max_cooling'] = parks_gdf[[f'cooling_diff_{d}m' for d in range(30, 301, 30)]].min(axis=1)

# Fix cooling_radius calculation
parks_gdf['cooling_radius'] = (
    parks_gdf[[f'cooling_diff_{d}m' for d in range(30, 301, 30)]]
    .idxmin(axis=1)
    .str.extract(r'(\d+)')  # Extract numeric part
    .iloc[:, 0]  # Ensure single column extraction
    .str.strip('m')  # Remove any stray 'm' characters
    .astype(float)  # Convert to float first to handle NaN
    .fillna(0)  # Replace NaN with 0
    .astype(int)  # Convert to int
)

# Verify results, focusing on Park 212
park_212 = parks_gdf[parks_gdf['park_id'] == 212].iloc[0]
print(f"Park 212: lst_park = {park_212['lst_park']:.1f}°C, max_cooling = {park_212['max_cooling']:.2f}°C, cooling_radius = {park_212['cooling_radius']}m")
print(parks_gdf[['park_id', 'lst_park', 'max_cooling', 'cooling_radius']].head())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress

# Scatter plot for max_cooling vs. NDVI and area
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# NDVI vs. max_cooling
slope_ndvi, intercept_ndvi, r_value_ndvi, p_value_ndvi, _ = linregress(parks_gdf['ndvi_mean'], parks_gdf['max_cooling'])
line_ndvi = slope_ndvi * parks_gdf['ndvi_mean'] + intercept_ndvi
ax1.scatter(parks_gdf['ndvi_mean'], parks_gdf['max_cooling'], color='#ADD8E6', alpha=0.5, s=50)
ax1.plot(parks_gdf['ndvi_mean'], line_ndvi, color='#1E90FF', label=f'R² = {r_value_ndvi**2:.2f}')
park_212_ndvi = parks_gdf[parks_gdf['park_id'] == 212]['ndvi_mean'].iloc[0]
park_212_cooling = parks_gdf[parks_gdf['park_id'] == 212]['max_cooling'].iloc[0]
ax1.scatter(park_212_ndvi, park_212_cooling, color='#4682B4', s=100, label='Park 212')
ax1.set_title('NDVI vs. Maximum Cooling Effect', fontsize=12)
ax1.set_xlabel('NDVI', fontsize=10)
ax1.set_ylabel('Maximum Cooling Effect (°C)', fontsize=10)
ax1.grid(linestyle='--', alpha=0.7)
ax1.legend()

# Area vs. max_cooling
slope_area, intercept_area, r_value_area, p_value_area, _ = linregress(parks_gdf['area_m2'], parks_gdf['max_cooling'])
line_area = slope_area * parks_gdf['area_m2'] + intercept_area
ax2.scatter(parks_gdf['area_m2']/10000, parks_gdf['max_cooling'], color='#ADD8E6', alpha=0.5, s=50)
ax2.plot(parks_gdf['area_m2']/10000, line_area, color='#1E90FF', label=f'R² = {r_value_area**2:.2f}')
park_212_area = parks_gdf[parks_gdf['park_id'] == 212]['area_m2'].iloc[0]/10000
ax2.scatter(park_212_area, park_212_cooling, color='#4682B4', s=100, label='Park 212')
ax2.set_title('Park Area vs. Maximum Cooling Effect', fontsize=12)
ax2.set_xlabel('Area (ha)', fontsize=10)
ax2.grid(linestyle='--', alpha=0.7)
ax2.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# Separate cooling and heat island parks
cooling_parks = parks_gdf[parks_gdf['max_cooling'] < 0]
heat_islands = parks_gdf[parks_gdf['max_cooling'] > 0]

# Calculate average values for key factors
factors = ['ndvi_mean', 'area_m2', 'impervious_prop', 'bare_sparse_prop', 'LSI']
cooling_avg = cooling_parks[factors].mean()
heat_avg = heat_islands[factors].mean()

# Perform t-test for NDVI to check significance
t_stat, p_value = ttest_ind(cooling_parks['ndvi_mean'], heat_islands['ndvi_mean'])
sig_note = f"p-value NDVI: {p_value:.3f}" if p_value < 0.05 else f"p-value NDVI: {p_value:.3f} (ns)"

# Create grouped bar chart with dual y-axes
x = np.arange(len(factors))
width = 0.35

fig, ax1 = plt.subplots(figsize=(10, 6))
bars1 = ax1.bar(x - width/2, [cooling_avg[f] for f in factors[:-1]] + [cooling_avg['area_m2']], width, 
                label='Cooling Parks (44%)', color='#ADD8E6')
bars2 = ax1.bar(x + width/2, [heat_avg[f] for f in factors[:-1]] + [heat_avg['area_m2']], width, 
                label='Heat Islands (56%)', color='#4682B4')

# Secondary y-axis for area_m2
ax2 = ax1.twinx()
ax2.set_ylim(ax1.get_ylim()[0] * 10000, ax1.get_ylim()[1] * 10000)  # Scale to m²
ax2.set_ylabel('Area (m²)', fontsize=10, color='#1E90FF')
ax1.set_ylabel('Average Value (Unitless)', fontsize=10)

# Customize plot
ax1.set_title('Contributing Factors: Cooling vs. Heat Island Parks', fontsize=12)
ax1.set_xticks(x)
ax1.set_xticklabels([f.replace('_m2', '') for f in factors], rotation=45, ha='right')
ax1.legend(loc='upper left')
ax1.grid(axis='y', linestyle='--', alpha=0.7)


ax1.text(0.5, -0.1, sig_note, transform=ax1.transAxes, ha='center', color='#1E90FF')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Extract Park 163 data
park_163 = parks_gdf.iloc[162]  # Index 162 for park_id 163
distances = range(0, 301, 30)
temps = [park_163['lst_park']] + [park_163[f'lst_buffer_{d}m'] for d in range(30, 301, 30)]

# Plot thermal profile
plt.figure(figsize=(10, 5))
plt.plot(distances, temps, marker='o', color='#4682B4', label='LST')
plt.title(f"Park 163: Thermal Profile\n(Estimated Area: ~20,000 m², NDVI: ~0.3)", fontsize=12)
plt.xlabel("Distance from Edge (m)", fontsize=10)
plt.ylabel("LST (°C)", fontsize=10)
plt.grid(linestyle='--', alpha=0.7)
max_dt = max(park_163[[f'cooling_diff_{d}m' for d in range(30, 301, 30)]])
max_dist = park_163[[f'cooling_diff_{d}m' for d in range(30, 301, 30)]].idxmax().split('_')[-1]
plt.annotate(f"Max ΔT: +{max_dt:.2f}°C at {max_dist}m", 
             xy=(int(max_dist), park_163[f'lst_buffer_{max_dist}m']), xytext=(100, 52),
             arrowprops=dict(facecolor='#1E90FF'))
plt.legend()
plt.show()

In [ ]:
park_163 = parks_gdf[parks_gdf['park_id'] == 163].iloc[0]
print(f"""
Park 163 Profile:
- Name: {park_163['FEATURE_ANAME']}
- Area: {park_163['area_m2']:,.0f} m²
- NDVI: {park_163['ndvi_mean']:.2f}
- LST: {park_163['lst_park']:.2f}
- Vegetation Cover: {park_163['vegetation_prop']:.0%}
- Impervious: {park_163['impervious_prop']:.0%}
- Max Coolin:{park_163['max_cooling']}
- Mx Radious: {park_163['cooling_radius']}
""")

In [ ]:
parks_gdf.columns